In [1]:
# import modules
import numpy as np
from gurobipy import *

In [2]:
# <----------------------- Read Data ----------------------->

path = './HarvestData.txt'

with open(path, 'r') as f:

    # Sets and indices
    F = list(range(int(f.readline()) + 1))  # fields
    H = list(range(1, int(f.readline()) + 1))  # harvesters
    K = list(range(1, int(f.readline()) + 1))  # harvest seasons
    T = list(range(int(f.readline()) + 1))  # time periods

    ## Parameters
    C_h = float(f.readline())/2  # rent cost: divided by 2 to make the conversion from per year to per harvest season
    if C_h <0.01:
        C_h += 1 # add a small epsilon to avoid redundant harvesters when c_h = 0
    C_o = float(f.readline())  # operational cost
    C_r = float(f.readline())  # transportation cost
    C_penalty = float(f.readline())  # penalty cost (derived from average gross profit)

    M = int(f.readline()) * np.ones(len(F))  # maximum number of harvesters that can harvest one field in a given period
    M = M.tolist()
    M_cap = float(f.readline())  # harvesting capacity

    loc = int(f.readline())  # facility location & 29 is the default facility location

    A = [0] + list(map(float, f.readline().split()))  # acres for fields & 29 is the default facility location
    R = {}  # ready date
    D = {}  # due date
    for k in K:
        R[0, k], D[0, k] = (1, 3)
    for i, ss in enumerate(f.readline().split(), start=1):
        for k in K:
            R[i, k] = int(ss)
    for i, ss in enumerate(f.readline().split(), start=1):
        for k in K:
            D[i, k] = int(ss)

    ins = {}
    I = np.zeros((len(F), len(K) + 1))  # indicator
    for k in K:
        ins[k] = list(map(int, f.readline().split()))
        for idx in ins[k]:
            I[idx, k] = 1 # indicate which fields need to be harvested during season k

    # distance matrix
    d = np.zeros((len(F), len(F)))
    for i in F[1:]:
        for j, ss in enumerate(f.readline().split(), start=1):
            d[i, j] = float(ss)

    for i in F[1:loc] + F[loc + 1:]:
        d[i][i] = 1000  # add an extra cost for staying in the same field except the facility 
                        #i.e. enforce the harvester not to stay in the same field except the facility

In [3]:
# Create the model

# solver settings
m = Model('Harvesting')
# m.params.LogFile = ('Harvest.log')
# setParam("LogToConsole", 0)
m.ModelSense = GRB.MINIMIZE
# m.Params.MIPGap = 0.01    # 1%
# m.Params.TimeLimit = 300  # 5 minutes

Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-19


In [4]:
# Variables

u = m.addVars([(h, k) for h in H for k in K], vtype=GRB.BINARY, name='u')
x = m.addVars([(f, t, k) for f in F for t in T for k in K], vtype=GRB.BINARY, name='x')
y = m.addVars([(h, t, i, j, k) for h in H for t in T for i in F for j in F for k in K], vtype=GRB.BINARY, name='y')
z = m.addVars([(h, f, t, k) for h in H for f in F for t in T for k in K], vtype=GRB.BINARY, name='z')
alpha = m.addVars([(h, f, t, k) for h in H for f in F for t in T for k in K], lb=0, ub=1, vtype=GRB.CONTINUOUS,
                  name='alpha')
s = m.addVars([(f, k) for f in F for k in K], lb=0, ub=len(T) + 1, vtype=GRB.INTEGER, name='s')
e = m.addVars([(f, k) for f in F for k in K], lb=0, ub=len(T) + 1, vtype=GRB.INTEGER, name='e')

p = m.addVars([(f, k) for f in F for k in K], lb=0, ub=len(T) + 1, vtype=GRB.CONTINUOUS, name='p')

In [5]:
# Objective function

m.setObjective(C_h * u.sum() +
               C_o * quicksum(x.sum(f, '*', '*') * A[f] for f in F) +
               quicksum(y.sum('*', '*', i, j, '*') * C_r * d[i][j] for i in F for j in F) +
               C_penalty * quicksum(p[f, k] * A[f] * I[f, k] for f in F for k in K) +
               quicksum((e[f, k] - s[f, k]) * I[f, k] for f in F for k in K)
               )

In [6]:
# Constraints

# m.update()

# Constraints for linearization of objective function
m.addConstrs(p[f, k] >= e[f, k] - D[f, k] for f in F for k in K)


# Constraints for harvesting
m.addConstrs(z.sum(h, '*', t, k) <= 1 for h in H for t in T for k in K)
m.addConstrs(z.sum('*', f, t, k) <= M[f] * x[f, t, k] for f in F for t in T for k in K)
m.addConstrs(z[h, f, t, k] >= alpha[h, f, t, k] for h in H for f in F for t in T for k in K)
m.addConstrs(z[h, f, t, k] <= alpha[h, f, t, k] + 0.999 for h in H for f in F for t in T for k in K)
m.addConstrs(z[h, f, t, k] <= u[h, k] for h in H for f in F for t in T for k in K)
m.addConstrs(z[h, f, t, k] <= x[f, t, k] for h in H for f in F for t in T for k in K)
m.addConstrs(x[f, t, k] <= z.sum('*', f, t, k) for f in F for t in T for k in K)

m.addConstrs(M_cap * u[h, k] >= quicksum(alpha.sum(h, f, '*', k) * A[f] for f in F) for h in H for k in K)
m.addConstrs(alpha.sum('*', f, '*', k) == I[f, k] for f in F for k in K)
m.addConstrs(x[f, t, k] == 0 for f in F for k in K for t in range(R[f, k]))
m.addConstrs(x[f, t, k] <= I[f, k] for f in F for k in K for t in T)
m.addConstrs(x.sum(f, '*', k) >= I[f, k] for f in F for k in K)



# Constraints for harvesting time period
m.addConstrs(R[f, k] <= s[f, k] for f in F for k in K)
m.addConstrs(s[f, k] <= t + len(T) * (1 - x[f, t, k]) for f in F for t in T for k in K)
m.addConstrs(t * x[f, t, k] <= e[f, k] for f in F for t in T for k in K)
m.addConstrs(s[f, k] <= e[f, k] for f in F for k in K)
m.addConstrs(x.sum(f, '*', k) <= e[f, k] - s[f, k] + 1 for f in F for k in K)



# Constraints for routing
m.addConstrs(u[h - 1, k] >= u[h, k] for h in H[1:] for k in K) # choose smaller harvesters' indices with priority

m.addConstrs(y.sum(h, 0, loc, '*', k) == u[h, k] for h in H for k in K)
m.addConstrs(y.sum(h, len(T) - 1, '*', loc, k) == u[h, k] for h in H for k in K)

m.addConstrs(y.sum(h, t - 1, '*', f, k) == y.sum(h, t, f, '*', k) for h in H for f in F for k in K for t in T[1:])

m.addConstrs(z[h, f, t, k] <= y.sum(h, t, f, '*', k) for h in H for f in F for t in T for k in K)
m.addConstrs(y.sum(h, t, '*', '*', k) <= u[h, k] for h in H for t in T for k in K)
m.addConstrs(y.sum(h, t, f, '*', k) <= I[f, k] for h in H for f in F[:loc] + F[loc + 1:] for t in T for k in K)
m.addConstrs(y.sum(h, t, loc, '*', k) <= 1 for h in H for t in T for k in K)

m.update()


In [7]:
# Solve the model
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 148995 rows, 769905 columns and 3757167 nonzeros
Model fingerprint: 0x7e92d004
Variable types: 19494 continuous, 750411 integer (749727 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 3e+04]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 140786 rows and 758975 columns (presolve time = 5s) ...
Presolve removed 147774 rows and 768239 columns
Presolve time: 7.95s
Presolved: 1221 rows, 1666 columns, 7161 nonzeros
Variable types: 245 continuous, 1421 integer (1406 binary)
Found heuristic solution: objective 786069.37100
Found heuristic solution: objective 736568.28100

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8986904e+05   1.800000e+01   0.000000e+00      9s
     287    4.0337356e+05   0.000000e+00   0

In [8]:
# Outputs


# output the optimal harvest time for each field
align = '{:^10} {:^10} {:^10} {:^10}'
# starting and ending period

print(align.format('Season', 'Field', 'Start_time', 'End_time', ' '))
for k in K:
    for f in ins[k]:
        print(align.format(k, f, int(s[f, k].x), int(e[f, k].x), ' '))
    print('\n')


# output the number of needed harvesters in each harvest seasons
align = '{:^10} {:^15}'
print(align.format('Season', '#_of_harvesters', ' '))
for k in K:
    num = 0
    for h in H:
        if u[h, k].x > 0.001:
            num += 1
    print(align.format(k, num, ' '))
    # print(k,' ', h)

print('\n')


# output how much to harvest in each field
align = '{:^10} {:^10} {:^10} {:^10} {:10}'
print(align.format('Season', 'Field', 'Harvester', 'Time', 'Proportion',' '))
align = '{:^10} {:^10} {:^10} {:^10} {:^10.2f}'
for k in K:
    for f in ins[k]:
        for h in H:
            for t in T:
                if alpha[h, f, t, k].x > 0.001:
                    print(align.format(k, f, h, t, alpha[h, f, t, k].x,' '))
    print('\n')



# output the machinery scheduling in this harvest plan
align = '{:^10} {:^10} {:^10} {:<10}'
print(align.format('Season', 'Harvester', 'Time', 'Route',' '))
for k in K:
    for h in H:
        for t in T:
            for i in F:
                for j in F:
                    if y[h, t, i, j, k].x > 0.001:
                        print(align.format(k, h,  t, str(i)+' --> '+str(j)))
    print('\n')


  Season     Field    Start_time  End_time 
    1          6          1          1     
    1          9          2          2     
    1          17         3          3     
    1          23         2          2     
    1          29         1          1     


    2          15         1          1     
    2          18         3          3     


    3          6          1          1     
    3          9          2          2     
    3          17         3          3     
    3          23         3          3     
    3          29         2          2     


    4          15         1          1     
    4          18         3          3     


    5          6          1          1     
    5          9          2          2     
    5          17         3          3     
    5          23         3          3     
    5          29         2          2     


    6          15         1          1     
    6          18         3          3     


    7          14   

In [9]:
# Running time & total cost

print('Running time: {:.2f}s'.format(m.Runtime))
print('Total cost($1000): {:.2f}'.format(m.objVal/1000))

Running time: 10.30s
Total cost($1000): 428.39


In [10]:
# Output costs


# rent cost
rent_cost = {}
rent_total = 0
for k in K:
    rent_cost[k] = 0
    for h in H:
        rent_cost[k] += C_h * u[h,k].x
    rent_total += rent_cost[k]
    rent_cost[k] = round(rent_cost[k]/1000,2)

print('Rent cost($1000): {} Total($1000): {:.2f}'.format(list(rent_cost.values()),rent_total/1000), end ='\n\n')



# operational cost
operation_cost = {}
operation_total = 0
for k in K:
    operation_cost[k] = 0
    for f in F:
        for t in T:
            operation_cost[k] += C_o * A[f]* x[f,t,k].x
    operation_total += operation_cost[k]
    operation_cost[k] = round(operation_cost[k]/1000, 2)

print('Operational cost($1000): {} Total($1000): {:.2f}'.format(list(operation_cost.values()),operation_total/1000), end='\n\n')



# transportation cost
tran_cost = {}
tran_total = 0
for k in K:
    tran_cost[k] = 0
    for h in H:
        for t in T:
            for i in F:
                for j in F:
                    tran_cost[k] += C_r * d[i][j]* y[h,t,i,j,k].x
    tran_total += tran_cost[k]
    tran_cost[k] = round(tran_cost[k]/1000, 2)

print('Transportion cost($1000): {} Total($1000): {:.2f}'.format(list(tran_cost.values()),tran_total/1000), end = '\n\n')



# penalty cost
penalty_cost = {}
penalty_total = 0
for k in K:
    penalty_cost[k] = 0
    for f in F:
        penalty_cost[k] += C_penalty * p[f,k].x*A[f]*I[f,k] 
    penalty_total += penalty_cost[k]
    penalty_cost[k] = round(penalty_cost[k]/1000, 2) 

print('Penalty cost($1000): {} Total($1000): {:.2f}'.format(list(penalty_cost.values()),penalty_total/1000), end = '\n\n')

Rent cost($1000): [50.0, 25.0, 50.0, 25.0, 50.0, 25.0, 25.0, 25.0, 25.0] Total($1000): 300.00

Operational cost($1000): [13.37, 6.39, 13.37, 6.39, 13.37, 6.39, 11.36, 8.5, 11.36] Total($1000): 90.47

Transportion cost($1000): [0.16, 0.12, 0.16, 0.12, 0.16, 0.12, 0.24, 0.2, 0.24] Total($1000): 1.49

Penalty cost($1000): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.21, 0.0, 18.21] Total($1000): 36.43

